In [1]:
import local_models.local_models
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
from importlib import reload
from ml_battery.utils import cmap
import matplotlib as mpl
import logging
import ml_battery.log
import time
import os
import functools
import collections
import itertools
import pymongo
import datetime
import pytz
import local_models.loggin
import local_models.tf_w2v_models
import pandas as pd
import pickle
from nltk.tokenize import TweetTokenizer
import re
import sklearn.base


logger = logging.getLogger(__name__)

np.random.seed(1)
#reload(local_models.loggin)
#reload(local_models.TLS_models)
np.warnings.filterwarnings('ignore')


/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [2]:
mpl.rcParams['figure.figsize'] = [8.0, 8.0]

In [3]:
DATETIME_FORMAT = "%a %b %d %H:%M:%S %z %Y"
def strptime(dt): return datetime.datetime.strptime(dt, DATETIME_FORMAT)
def strftime(dt): return dt.strftime(DATETIME_FORMAT)

In [4]:
RUN = 1
BANDWIDTH = 0.35
SEED = 1
np.random.seed(SEED)
KERNEL=local_models.local_models.GaussianKernel
root_dir = "/home/scott/local_w2v/"
project_dir = os.path.join(root_dir, "r{:03d}_k{}".format(RUN, KERNEL(bandwidth=BANDWIDTH)))
data_dir = os.path.join(root_dir, "data")
os.makedirs(project_dir, exist_ok=1)

In [5]:
class fuckit(object):
    def __init__(self, message=None):
        self.message = message
    def __enter__(self): return self
    def __exit__(self, *args):
        if self.message is not None:
            print(self.message)
        return True

In [6]:
with fuckit("nothing to del"):
    TFGlobalSesh.close()
with fuckit("nothing to del"):
    del TFGlobalSesh 
with fuckit("nothing to del"):
    with TFGlobalGraph.as_default():
        tf.reset_default_graph()
with fuckit("nothing to del"):
    del TFGlobalGraph 
with fuckit("nothing to del"):
    del model

nothing to del
nothing to del
nothing to del
nothing to del
nothing to del


In [7]:
VOCAB_SIZE = 6000
BASE_MODEL_EPOCHS = 10000
BASE_MODEL_RUN = 2

In [8]:
try:
    with open(os.path.join(root_dir, "base_model_v{:08d}_e{:08d}_r{:03d}.model".format(VOCAB_SIZE, BASE_MODEL_EPOCHS, BASE_MODEL_RUN)), 'rb') as f:
        model = pickle.load(f)
except:
    raise Exception("no such model")

In [9]:
model.sort("sweet")

['sweet',
 'yummy',
 'lovely',
 'delicious',
 'nice',
 'beautiful',
 'cool',
 'gorgeous',
 'tasty',
 'cute',
 'funny',
 'rare',
 'awesome',
 'wet',
 'fabulous',
 'adorable',
 'sick',
 'goodnight',
 'similar',
 'wonderful',
 'familiar',
 'neat',
 'sad',
 'weird',
 'perfect',
 'good',
 'happy',
 'sexy',
 'welcome',
 'rad',
 'warm',
 'bff',
 'strange',
 'goood',
 'ricky',
 'bright',
 'dope',
 'gd',
 'fly',
 'romantic',
 'lonely',
 'boring',
 'aww',
 'lucky',
 'badass',
 'jealous',
 'nasty',
 'fresh',
 'windy',
 'sleepy',
 'glorious',
 'amazing',
 'impressive',
 'xoxo',
 'creepy',
 'chill',
 'salad',
 'disgusting',
 'honey',
 'cheap',
 'scary',
 'awww',
 'loves',
 'proud',
 'wicked',
 'lame',
 'clever',
 'mmm',
 'hot',
 'dry',
 'spicy',
 'bomb',
 'hilarious',
 'priceless',
 'tgif',
 'friendly',
 'easy',
 'pleased',
 'blessed',
 'painful',
 'loving',
 'safe',
 'dangerous',
 'fantastic',
 'helpful',
 'alright',
 'silly',
 'annoying',
 'stoked',
 'kisses',
 'dirty',
 'fab',
 'chilly',
 'incre

In [10]:
model.sess.run(model.model.biases)

array([ 9.264237  , -1.5266105 , -0.19153859, ..., -0.3512247 ,
       -0.6663339 ,  1.1241772 ], dtype=float32)

In [11]:
shooting_data = pd.read_csv(os.path.join(root_dir, "shootings_tweets.csv"))

In [12]:
def remove_by_regex(text, regexp):
    return regexp.sub("", text)

url_re = re.compile(r"http.?://[^\s]+[\s]?")
def remove_urls(text):
    return remove_by_regex(text, url_re)

specialchar_re = re.compile("[" + ''.join(map(re.escape, 
    [",", ":", "\"", "=", "&", ";", "%", "$", "@", "%", "^", "*", "(", ")", "{", "}",
     "[", "]", "|", "/", "\\", ">", "<", "-", "!", "?", ".", "'", "--", "---"])) + "]")
def remove_special_chars(text):
    return remove_by_regex(text, specialchar_re)

username_re = re.compile(r"@[^\s]+[\s]?")
def remove_usernames(text):
    return remove_by_regex(text, username_re)

number_re = re.compile(r"\s?[0-9]+\.?[0-9]*")
def remove_numbers(text):
    return remove_by_regex(text, number_re)
    
def lower(text):
    return text.lower()

cleanup = (remove_urls,
           remove_usernames,
           remove_special_chars,
           remove_numbers,
           lower)


def count(iterable, item):
    tot = 0
    for i in iterable:
        if i == item:
            tot += 1
    return tot

In [13]:
tknzr = TweetTokenizer(strip_handles=True, reduce_len=True) 

In [14]:
tweets = []
for tweet in shooting_data[shooting_data["collection"] == "christchurch_shooting"]["text"]:
    for cleaner in cleanup:
        tweet = cleaner(tweet)
    tweets.append(tweet)
        
print(len(tweets), count(tweets, None))
tweets = [i for i in tweets if i is not None]
print(len(tweets))
toknized_tweets = [tknzr.tokenize(t) for t in tweets]

17569 0
17569


In [15]:
toknized_tweets[0]

['it', 'would', 'seem', 'a', 'shame', 'to', 'buck', 'tradition']

In [16]:
first_tweet = datetime.datetime.fromtimestamp(shooting_data[shooting_data["collection"] == "christchurch_shooting"]["timestamp"].min())
last_tweet = datetime.datetime.fromtimestamp(shooting_data[shooting_data["collection"] == "christchurch_shooting"]["timestamp"].max())
first_tweet, last_tweet, last_tweet - first_tweet

(datetime.datetime(2019, 3, 7, 17, 6, 41),
 datetime.datetime(2019, 3, 15, 17, 40, 9),
 datetime.timedelta(8, 2008))

In [17]:
BANDWIDTH = 24*60*60 #1 day
kernel = local_models.local_models.TriCubeKernel(bandwidth=BANDWIDTH)
EPOCHS = 3000
modl = sklearn.base.clone(model)
modl.epochs = EPOCHS

In [20]:
model.sess

In [19]:
modl.sess

In [18]:
modl.sess.run(modl.model.biases)

FailedPreconditionError: Attempting to use uninitialized value w2v/variables/Variable_2
	 [[{{node _retval_w2v/variables/Variable_2_0_0}}]]

In [ ]:
w2v_models = local_models.local_models.LocalModels(modl)

In [ ]:
datafied_toknized_tweets = local_models.tf_w2v_models.build_dataset_predictionary(toknized_tweets, model.dictionary)

In [ ]:
w2v_models.fit(datafied_toknized_tweets, 
    model_cluster_features=shooting_data[
        shooting_data["collection"] == "christchurch_shooting"
    ]["timestamp"].values.reshape(-1,1)) 

In [ ]:
model.final_embeddings[model.dictionary["shooting"]]
def shooting_vec_extractor(m,q,x,y,w):
    return m.final_embeddings[m.dictionary["shooting"]]

In [ ]:
query = np.linspace(first_tweet.timestamp(), last_tweet.timestamp(), 5000).reshape(-1,1)

In [ ]:
shooting_vec_over_time = w2v_models.transform(query, kernel=kernel, r=kernel.support_radius(), model_postprocessor=shooting_vec_extractor, parallel=True)